In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from datasets import load_dataset, load_dataset_builder

/home/walml/miniforge3/envs/zoobot39_cu121_dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
sys.path.append('../..')

from baselines.galaxy10_decals.dataset_wrapper import DatasetWrapper

In [4]:
# I assume this will eventually be on huggingface and downloaded directly

# !wget https://www.astro.utoronto.ca/~hleung/shared/Galaxy10/Galaxy10_DECals.h5 -O ../../scripts/gz10/Galaxy10_DECals.h5

In [5]:
from scripts.gz10 import build_parent_sample

# TODO replace with your paths if you prefer
build_parent_sample.save_in_standard_format(
    input_path='../../scripts/gz10/Galaxy10_DECals.h5',
    output_dir='../../scripts/gz10/datafiles'  # expected location when running with __main__ in build_parent_sample.py
)
# takes around 10 minutes

Found 766 unique HEALPix indices


Processing HEALPix indices: 100%|██████████| 766/766 [00:00<00:00, 4353437.49it/s]


In [26]:
# relative import as scripts not part of astropile package

# currently fails as no object_id key in original Galaxy10_DECals.h5 specified to load
dataset = load_dataset('../../scripts/gz10/gz10.py', name="gz10_images", trust_remote_code=True)  

Generating train split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [27]:
# not sure why but I need to unpack files variable to avoid "train has no data" error
# Instruction "train" corresponds to no data!
dataset = load_dataset('../../scripts/gz10/gz10.py', name="gz10_with_healpix_with_images", trust_remote_code=True)


Generating train split: 17736 examples [00:07, 2499.99 examples/s]


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['gz10_label', 'ra', 'dec', 'object_id', 'images', 'pixel_scale'],
        num_rows: 17736
    })
})

In [7]:
# dataset['train']['images'][0].shape

In [8]:

# this runs successfully (though since it has no images I won't use it further here)
# dataset = load_dataset('../../scripts/gz10/gz10.py', name="gz10_with_healpix", trust_remote_code=True)


In [9]:
# currently has everything in 'train' split
dataset_dict_with_splits = dataset['train'].train_test_split(test_size=0.3)

In [10]:

decals10_train = dataset_dict_with_splits['train']
decals10_test = dataset_dict_with_splits['test']

In [11]:

decals10_train.set_format('torch')
decals10_test.set_format('torch')

In [12]:
decals10_test

Dataset({
    features: ['gz10_label', 'ra', 'dec', 'object_id', 'images', 'pixel_scale'],
    num_rows: 5321
})

In [13]:
len(decals10_train)

12415

In [14]:
decals10_train[0]['gz10_label']

tensor(2)

In [15]:
decals10_train[0]['images'].shape

torch.Size([256, 256, 3])

In [16]:
# # takes about 5 minutes (strangely?)

# def channels_first_to_channels_last(example):
#     example["images"] =  example['images'].transpose(2, 0)
#     return example

# # dataset wrapper assumes channels last (pytorch convention) for e.g. taking a feature mean
# decals10_train = decals10_train.map(channels_first_to_channels_last)
# decals10_test = decals10_test.map(channels_first_to_channels_last)

# decals10_train[0]['images'].shape

In [17]:
wrapper = DatasetWrapper(
    train_dataset=decals10_train,
    test_dataset=decals10_test,
    feature_flag='images',
    label_flag='gz10_label',
    loading='iterated'
)

In [18]:
wrapper.prepare_data()
wrapper.feature_mean, wrapper.feature_std

(tensor([[[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 
         [[nan]],
 


In [19]:
wrapper.feature_mean.shape

torch.Size([256, 1, 1])

In [20]:
train_loader = wrapper.train_dataloader()
val_loader = wrapper.val_dataloader()
test_loader = wrapper.test_dataloader()

In [22]:
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([128, 256, 256, 3]) torch.Size([128])


In [21]:
im_np = images[0].numpy().transpose(1, 2, 0)
im_np.min(), im_np.max()

(-0.11923878, 2.390703)

In [23]:
import galaxy10_model

config = galaxy10_model.default_config()


model = galaxy10_model.SmallConvModel(config)

In [24]:
trainer = pl.Trainer(max_epochs=1, accelerator='gpu', logger=False, enable_checkpointing=False)


trainer.fit(model, wrapper)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params
------------------------------------------------------
0 | encoder        | Sequential         | 19.4 K
1 | head           | Sequential         | 1.4 M 
2 | model          | Sequential         | 1.4 M 
3 | train_accuracy | MulticlassAccuracy | 0     
4 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.618     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[128, 256, 256, 3] to have 3 channels, but got 256 channels instead

In [29]:
# alternative non-lightning loading option

In [30]:
# target_size = 64
# from torchvision.transforms import Compose, Normalize, Resize, Lambda, RandomHorizontalFlip, RandomVerticalFlip


# normalize = Normalize(mean=1., std=1.)
#                     #   https://pytorch.org/vision/main/generated/torchvision.transforms.Lambda.html)

# _transforms = Compose(
#     [
#         Resize(target_size),
#         RandomHorizontalFlip(),
#         RandomVerticalFlip(),
#         Lambda(func=lambda x: np.arcsinh(x)),
#         normalize
#     ]
# )

# def transforms(examples):
#     examples["pixel_values"] = [_transforms(image.convert("RGB")) for image in examples["image"]]
#     return examples